**Import Required Packages**

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from google.colab import files
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

**Load datasets**

In [2]:
# Upload the dataset
file_path = files.upload()

# Extract the uploaded file name
uploaded_file_name = list(file_path.keys())[0]

# Load the dataset with appropriate encoding
try:
    df = pd.read_csv(uploaded_file_name, delimiter=",", encoding="utf-8")
except UnicodeDecodeError:
    # If utf-8 fails, try 'latin-1' encoding
    df = pd.read_csv(uploaded_file_name, delimiter=",", encoding="latin-1")

# Basic info about the dataset
df.info()

Saving football_matches.csv to football_matches.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070 entries, 0 to 4069
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     4070 non-null   int64  
 1   season                4070 non-null   object 
 2   date                  4070 non-null   object 
 3   home_team             4070 non-null   object 
 4   away_team             4070 non-null   object 
 5   home_clearances       4070 non-null   int64  
 6   home_corners          4070 non-null   int64  
 7   home_fouls_conceded   4070 non-null   int64  
 8   home_offsides         4070 non-null   int64  
 9   home_passes           4070 non-null   int64  
 10  home_possession       4070 non-null   float64
 11  home_red_cards        4070 non-null   int64  
 12  home_shots            4070 non-null   int64  
 13  home_shots_on_target  4070 non-null   int64  
 14  home_tackles        

**Data Preprocessing**

In [3]:
df.dropna(inplace=True)

# Encode categorical variables
encoder_home = LabelEncoder()
encoder_away = LabelEncoder()
df['home_team'] = encoder_home.fit_transform(df['home_team'])
df['away_team'] = encoder_away.fit_transform(df['away_team'])
df['result'] = LabelEncoder().fit_transform(df['result'])

features = ['home_team', 'away_team', 'home_clearances', 'away_clearances', 'home_corners', 'away_corners',
            'home_fouls_conceded', 'away_fouls_conceded', 'home_shots', 'away_shots', 'goal_home_ft', 'goal_away_ft']
X = df[features]
y = df['result']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

**Building a Pandas Dataframer**

In [4]:
# ------------------- Building a Pandas Dataframer -------------------

batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

**Designing the Discriminative Model**

In [8]:
# ------------------- Designing the Discriminative Model -------------------

# PyTorch Model
class RandomForestInspiredNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RandomForestInspiredNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Initialize PyTorch Model
input_size = X_train.shape[1]
num_classes = len(np.unique(y))
model = RandomForestInspiredNN(input_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

**Validation and Testing**

In [10]:
# ------------------- Validation and Testing -------------------

def evaluate_model(model, loader, phase):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch_X, batch_y in loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(batch_y.tolist())
            y_pred.extend(predicted.tolist())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"\n{phase} Classification Report:\n", classification_report(y_true, y_pred))
    print(f"{phase} Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

    return accuracy, precision, recall, f1

# Evaluate before training
evaluate_model(model, test_loader, "Before Training")


Before Training Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.94      0.60       361
           1       0.33      0.05      0.09       250
           2       0.10      0.00      0.01       203

    accuracy                           0.43       814
   macro avg       0.29      0.33      0.23       814
weighted avg       0.32      0.43      0.30       814

Before Training Confusion Matrix:
 [[339  13   9]
 [237  13   0]
 [189  13   1]]


(0.43366093366093367,
 0.3238401503107386,
 0.43366093366093367,
 0.29701093629690933)

**Hyperparameter Tuning**

In [11]:
# ------------------- Hyperparameter Tuning -------------------

from itertools import product

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'batch_size': [16, 32, 64],
    'num_epochs': [30, 50]
}

best_accuracy = 0
best_params = {}

# Iterate over all possible hyperparameter combinations
for lr, batch_size, num_epochs in product(param_grid['learning_rate'], param_grid['batch_size'], param_grid['num_epochs']):

    # Update DataLoader with the new batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define a new model for each combination
    model = RandomForestInspiredNN(input_size, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'LR: {lr}, Batch Size: {batch_size}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate model
    accuracy, precision, recall, f1 = evaluate_model(model, test_loader, f"LR: {lr}, Batch Size: {batch_size}, Epochs: {num_epochs}")

    # Track the best hyperparameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'learning_rate': lr, 'batch_size': batch_size, 'num_epochs': num_epochs}

# Print the best hyperparameters
print("\nBest Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

# Train final model with best hyperparameters
final_lr = best_params['learning_rate']
final_batch_size = best_params['batch_size']
final_epochs = best_params['num_epochs']

# Update DataLoader with the best batch size
train_loader = DataLoader(train_dataset, batch_size=final_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=final_batch_size, shuffle=False)

# Define the final model
final_model = RandomForestInspiredNN(input_size, num_classes)
final_optimizer = optim.Adam(final_model.parameters(), lr=final_lr, weight_decay=1e-4)
final_criterion = nn.CrossEntropyLoss()

# Training the final model
for epoch in range(final_epochs):
    final_model.train()
    for batch_X, batch_y in train_loader:
        final_optimizer.zero_grad()
        outputs = final_model(batch_X)
        loss = final_criterion(outputs, batch_y)
        loss.backward()
        final_optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Final Model - Epoch [{epoch+1}/{final_epochs}], Loss: {loss.item():.4f}')

# Final evaluation
evaluate_model(final_model, test_loader, "Final Model")

LR: 0.001, Batch Size: 16, Epoch [10/30], Loss: 0.0001
LR: 0.001, Batch Size: 16, Epoch [20/30], Loss: 0.0000
LR: 0.001, Batch Size: 16, Epoch [30/30], Loss: 0.2567

LR: 0.001, Batch Size: 16, Epochs: 30 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       361
           1       1.00      1.00      1.00       250
           2       1.00      1.00      1.00       203

    accuracy                           1.00       814
   macro avg       1.00      1.00      1.00       814
weighted avg       1.00      1.00      1.00       814

LR: 0.001, Batch Size: 16, Epochs: 30 Confusion Matrix:
 [[361   0   0]
 [  0 250   0]
 [  0   0 203]]
LR: 0.001, Batch Size: 16, Epoch [10/50], Loss: 0.0001
LR: 0.001, Batch Size: 16, Epoch [20/50], Loss: 0.0000
LR: 0.001, Batch Size: 16, Epoch [30/50], Loss: 0.0001
LR: 0.001, Batch Size: 16, Epoch [40/50], Loss: 0.0000
LR: 0.001, Batch Size: 16, Epoch [50/50], Loss: 0.0000

LR: 0.001, B

(1.0, 1.0, 1.0, 1.0)